####
### <font color="GREEN">Introduction to Web Scraping</font>
# <font color="RED">Amazon Reviews Scraping</font>

In [32]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [33]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [34]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.in/product-reviews/B075P5L5D4/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1#reviews-filter-bar'

In [35]:
# Define Page No
len_page = 100

### <font color="red">Functions</font>

In [36]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):

    # Empty List define to store all pages html data
    soups = []



    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):

        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }

        # Request make for each page
        response = requests.get(url, headers=headers)

        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')

        # Add single Html page data in master soups list
        soups.append(soup)

    return soups

In [37]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []

    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')

    # Iterate all Reviews BOX
    for box in boxes:

        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            print(datetime_str)
            date = datetime.strptime(datetime_str, '%d %B %Y').strftime("%d/%m/%Y")
        except Exception as e:
            print(e)
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'



        # create Dictionary with al review data
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)

    return data_dicts

### <font color="red">Data Process</font>

In [38]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [39]:
# Empty List to Hold all reviews data
reviews = []

In [ ]:
# Iterate all Html page
for html_data in html_datas:

    # Grab review data
    review = getReviews(html_data)

    # add review data in reviews empty list
    reviews += review

In [41]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [45]:
df_reviews

,Name,Stars,Title,Date,Description
0,SAYYED USMAN AHMED,5.0,5.0 out of 5 stars\nQuality,17/02/2024,Great work
1,Misty gadhvi,5.0,5.0 out of 5 stars\nGood,15/02/2024,Good
2,Abhinav Priyam,5.0,5.0 out of 5 stars\nProduct details,01/02/2024,Products was very nice and the finishing of th...
3,Chirag bhuva,1.0,1.0 out of 5 stars\nWest product,14/01/2024,The media could not be loaded.\n ...
4,Amazon Customer,4.0,4.0 out of 5 stars\nNice watch,08/01/2024,
...,...,...,...,...,...
995,Jitendra Pradhan,1.0,1.0 out of 5 stars\nDefective,04/01/2024,I have got a defective item it is not running ...
996,MAULIK UPLA,5.0,5.0 out of 5 stars\nNice Watch,31/12/2023,Watch is nice. It is multifunctional watch.Loo...
997,Babul S.,5.0,5.0 out of 5 stars\nvalue for money,24/12/2023,Its value for money product
998,Animesh Swain,5.0,5.0 out of 5 stars\nClassy Watch,20/12/2023,So Elegant....So Beautiful.....Just Looking Li...


In [44]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)